In [1]:
import pandas as pd
import pyreadr

# Specify the path to the ALS_progression_rate rds file
file_path = 'ALS_progression_rate.1822x370.rds'

# Read the rds file into a pandas dataframe
r = pyreadr.read_r(file_path)
df = r[None]


In [2]:
df.rename(columns={'dFRS': 'response'}, inplace=True)


In [3]:
# Split the data into training set and prediction set
train_set = df[df['response'].notna()]
prediction_set = df[df['response'].isna()]

print(train_set.shape)
print(prediction_set.shape)


(1622, 370)
(200, 370)


In [4]:
from sklearn.model_selection import train_test_split

# Assuming df is your DataFrame, 'response' is your target variable

# Split the data into training set and test set
train, test  = train_test_split(train_set, test_size=0.2, random_state=42)

# Split the data into training+validation set and test set
X = train_set.drop('response', axis=1)
y = train_set['response']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Split the training+validation set into training and validation sets

pycaret from here

In [ ]:
from pycaret.regression import *
s = setup(data = train_set, target ='response',train_size=0.8)

In [ ]:
best = compare_models()

In [ ]:
predict_model(best, data=prediction_set)

lazypredict here

In [6]:
from lazypredict import LazyRegressor
import numpy as np

In [7]:
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)

 21%|██▏       | 9/42 [00:12<00:56,  1.70s/it]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfGammaLoss'.


 71%|███████▏  | 30/42 [00:25<00:03,  4.00it/s]

PoissonRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfPoissonLoss'.


 98%|█████████▊| 41/42 [00:41<00:00,  1.36it/s]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004112 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 28252
[LightGBM] [Info] Number of data points in the train set: 1297, number of used features: 338
[LightGBM] [Info] Start training from score -0.697527


100%|██████████| 42/42 [00:41<00:00,  1.01it/s]


In [8]:
print(models)

                                                      Adjusted R-Squared  \
Model                                                                      
Lars                          373644788190158921494857795071006212096.00   
TransformedTargetRegressor                 15059808942524347258503168.00   
LinearRegression                           15059808942524347258503168.00   
RANSACRegressor                             3221299539381674905174016.00   
SGDRegressor                                                    18011.54   
KernelRidge                                                        18.04   
GaussianProcessRegressor                                           17.17   
MLPRegressor                                                       14.71   
ExtraTreeRegressor                                                 14.70   
PassiveAggressiveRegressor                                         13.23   
DecisionTreeRegressor                                              12.49   
LinearSVR   

and auto-sklearn, guess were going to loonix

In [6]:
from pprint import pprint

import sklearn.datasets
import sklearn.metrics

import autosklearn.regression
import matplotlib.pyplot as plt

In [10]:
automl = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=120, per_run_time_limit=30, memory_limit=6000)
automl.fit(X_train, y_train)

AutoSklearnRegressor(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                     memory_limit=6000, per_run_time_limit=30,
                     time_left_for_this_task=120)

In [12]:
print(automl.leaderboard())

          rank  ensemble_weight           type      cost   duration
model_id                                                           
2            1             0.72  random_forest  0.817380  19.335031
13           2             0.24    extra_trees  0.828724   3.339578
4            3             0.04  liblinear_svr  0.941059   0.314204


In [13]:
from sklearn.metrics import mean_squared_error

train_pred = automl.predict(X_train)
test_pred = automl.predict(X_test)

print("Train RMSE:", mean_squared_error(y_train, train_pred, squared=False))
print("Test RMSE:", mean_squared_error(y_test, test_pred, squared=False))

Train RMSE: 0.3619818565818878
Test RMSE: 0.5234378581724037


In [17]:

from sklearn.metrics import mean_squared_error

automl = autosklearn.regression.AutoSklearnRegressor(
	time_left_for_this_task=120,
	per_run_time_limit=30,
	memory_limit=6000,
	tmp_folder="/tmp/autosklearn_resampling_example_tmp",
	disable_evaluator_output=False,
	resampling_strategy="cv",
	resampling_strategy_arguments={"folds": 10},
)
automl.fit(X_train, y_train)

# One can use models trained during cross-validation directly to predict
# for unseen data. For this, all k models trained during k-fold
# cross-validation are considered as a single soft-voting ensemble inside
# the ensemble constructed with ensemble selection.
print("Before re-fit")
predictions = automl.predict(X_test)
print("RMSE score CV", mean_squared_error(y_test, predictions, squared=False))

[WARNING] [2024-04-12 09:32:36,114:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2024-04-12 09:32:38,087:Client-EnsembleBuilder] No runs were available to build an ensemble from
Before re-fit
RMSE score CV 0.5147098301256179


In [19]:
print("After re-fit")
automl.refit(X_train.copy(), y_train.copy())
predictions = automl.predict(X_test)
print("RMSE score CV", mean_squared_error(y_test, predictions, squared=False))


After re-fit
RMSE score CV 0.5172109555744472


In [ ]:
scorer =autosklearn.metrics.root_mean_squared_error

automl = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=3600*2,  # Increase time limit
    per_run_time_limit=1800,  # Increase time limit per run
    memory_limit=8000,
    tmp_folder="/tmp/autosklearn_resampling_example_tmp",
    disable_evaluator_output=False,
    resampling_strategy="cv",
    resampling_strategy_arguments={"folds": 20,"train_size":0.95,"shuffle":True},
	metric=scorer  # Increase number of folds
)

automl.fit(X_train, y_train)



#print("After re-fit")
#automl.refit(X_train.copy(), y_train.copy())
predictions = automl.predict(X_test)
print("RMSE score CV", mean_squared_error(y_test, predictions, squared=False))

In [9]:

from sklearn.metrics import mean_squared_error
predictions = automl.predict(X_test)
print("RMSE score CV", mean_squared_error(y_test, predictions, squared=False))

RMSE score CV 0.5033208483401808


In [28]:
import numpy as np
#Get the CV results
cv_results = automl.cv_results_

# Get the mean test score for each model
mean_test_scores = cv_results['mean_test_score']

# Convert to RMSE (as the scores are negated mean squared errors)
rmses = np.sqrt(-mean_test_scores)

# Get the mean RMSE
mean_rmse = np.mean(rmses)

print("Expected error (mean CV RMSE): ", mean_rmse)

Expected error (mean CV RMSE):  nan


/tmp/ipykernel_17322/2221981772.py:9: RuntimeWarning: invalid value encountered in sqrt
  rmses = np.sqrt(-mean_test_scores)


In [22]:
automl.leaderboard()


,rank,ensemble_weight,type,cost,duration
model_id,,,,,
77,1,0.04,extra_trees,0.817759,44.287058
69,2,0.22,extra_trees,0.817772,45.218471
73,3,0.20,adaboost,0.831001,71.883102
67,4,0.30,gradient_boosting,0.850387,7.990810
4,5,0.02,liblinear_svr,0.903092,1.878924
41,6,0.04,gradient_boosting,0.942080,98.599346
15,7,0.04,libsvm_svr,0.959778,85.107737
61,8,0.14,decision_tree,0.975067,2.027059
